# Airport Charging Project

In [93]:
import pandas as pd
import math
import numpy as np

# Columns to keep
airport_cols = ['Site Id', 'Name', 'State Name', 'City', 'ARP Latitude', 
                'ARP Longitude', 'NPIAS Hub', 'NPIAS Role']
charger_cols = ['State','City','Latitude', 'Longitude']

# Import data w/ specified columns only
airports = pd.read_csv('downloaded_data/airport-data.csv')[airport_cols]
chargers = pd.read_csv('downloaded_data/dc-chargers-data.csv')[charger_cols]
states_xref = pd.read_csv('downloaded_data/states_xref.csv').set_index('Code').to_dict()

# Rename some columns
airport_cols_new = ['Site_ID', 'Name', 'State', 'City', 'ARP_Latitude', 
                    'ARP_Longitude', 'NPIAS_Hub', 'NPIAS_Role']
charger_cols_new = ['State_abv', 'City','CH_Latitude', 'CH_Longitude']
airports.columns = airport_cols_new
chargers.columns = charger_cols_new

C:\Users\Philip Harman\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3156: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [94]:
# Reformat ARP_Latitude + ARP_Longitude
airports['ARP_Latitude'] = airports.ARP_Latitude.apply(
    lambda x: int(x[:3]) + int(x[4:6])/60 + float(x[7:-1])/3600)
airports['ARP_Longitude'] = airports.ARP_Longitude.apply(
    lambda x: -1*(int(x[:3]) + int(x[4:6])/60 + float(x[7:-1])/3600))

# NPIAS_Hub changes: Drop where NPIAS_Hub is blank
NPIAS_Category = []
Hubs = airports.NPIAS_Hub.dropna().unique()
Roles = airports.NPIAS_Role.dropna().unique()
for i in range(len(airports)):
    if airports.NPIAS_Hub[i] not in Hubs:
        if airports.NPIAS_Role[i] not in Roles:
            NPIAS_Category.append('Drop')
        else:
            NPIAS_Category.append('Reliever')
    else:
        NPIAS_Category.append(airports.NPIAS_Hub[i])
airports['NPIAS_Category'] = NPIAS_Category
airports = airports[airports.NPIAS_Category != 'Drop']

# Drop unneeded NPIAS Columns
airports = airports.drop(columns = ['NPIAS_Hub','NPIAS_Role'])

# Change Airport name and City/State column to upper case
airports.Name = airports.Name.apply(lambda x: str(x).title())
airports.City = airports.City.apply(lambda x: str(x).title())
airports.State = airports.State.apply(lambda x: str(x).title())
print(airports.shape)
airports.head(20)

(3180, 7)


,Site_ID,Name,State,City,ARP_Latitude,ARP_Longitude,NPIAS_Category
0,50009.*A,Adak,Alaska,Adak Island,51.883583,-176.642483,Reliever
1,50016.1*A,Akhiok,Alaska,Akhiok,56.938692,-154.182556,Reliever
2,50017.*A,Akiachak,Alaska,Akiachak,60.913808,-161.493331,Reliever
3,50020.*A,Akiak,Alaska,Akiak,60.902897,-161.230583,Reliever
4,50022.1*A,Akutan,Alaska,Akutan,54.144611,-165.604108,Reliever
5,50024.1*A,Alakanuk,Alaska,Alakanuk,62.683056,-164.722194,Reliever
6,50027.53*A,Aleknagik /New,Alaska,Aleknagik,59.282556,-158.617867,Reliever
7,50029.1*A,Allakaket,Alaska,Allakaket,66.551833,-152.622167,Reliever
9,50029.61*A,Ambler,Alaska,Ambler,67.106200,-157.857156,Reliever
11,50032.*A,Anaktuvuk Pass,Alaska,Anaktuvuk Pass,68.133611,-151.743333,Reliever


In [95]:
# Rename states in Charger DF to be the full state name (to match Airport DF)
chargers['State'] = chargers.State_abv.apply(lambda x: states_xref['State'][x])

# Export CSV
chargers.to_csv('prepped_data/chargers_prepped.csv') # all chargers

chargers = chargers.drop(columns = ['State_abv', 'City'])
chargers.head()

,CH_Latitude,CH_Longitude,State
0,32.367916,-86.267021,Alabama
1,33.745843,-84.398837,Georgia
2,33.821911,-84.367461,Georgia
3,33.760256,-84.543822,Georgia
4,29.779918,-95.435385,Texas


In [96]:
#Left join Airports with Chargers where States are equal
Joined_Set=pd.merge(airports,chargers,left_on='State',right_on='State',how='left')

#Convert lat/longs to float
Joined_Set.ARP_Latitude.astype(float)
Joined_Set.ARP_Longitude.astype(float)
Joined_Set.CH_Latitude.astype(float)
Joined_Set.CH_Longitude.astype(float)
Joined_Set.head()

,Site_ID,Name,State,City,ARP_Latitude,ARP_Longitude,NPIAS_Category,CH_Latitude,CH_Longitude
0,50009.*A,Adak,Alaska,Adak Island,51.883583,-176.642483,Reliever,56.469252,-132.379680
1,50009.*A,Adak,Alaska,Adak Island,51.883583,-176.642483,Reliever,55.477123,-133.133806
2,50009.*A,Adak,Alaska,Adak Island,51.883583,-176.642483,Reliever,61.178048,-149.916588
3,50009.*A,Adak,Alaska,Adak Island,51.883583,-176.642483,Reliever,60.501690,-151.077002
4,50009.*A,Adak,Alaska,Adak Island,51.883583,-176.642483,Reliever,58.300164,-134.408460


In [97]:
#Define lat/long distance calculation function
def Get_Distance(lat1,lon1,lat2,lon2):
    dlon=math.radians(lon2)-math.radians(lon1)
    dlat=math.radians(lat2)-math.radians(lat1)
    a=((math.sin(dlat/2))**2)+\
        math.cos(math.radians(lat1))*math.cos(math.radians(lat2))*(math.sin(dlon/2))**2
    c=2*math.atan2(math.sqrt(a),math.sqrt(1-a))
    d=3961*c
    return d

In [98]:
#Apply function to calculate distance from all chargers to all airports
Joined_Set['Distance']=Joined_Set.apply(lambda row : Get_Distance(row.ARP_Latitude,\
                                                     row.ARP_Longitude,\
                                                     row.CH_Latitude,\
                                                     row.CH_Longitude)\
                                                    ,axis=1)

In [99]:
print(Joined_Set.shape)
Joined_Set.head()

(1404575, 10)


,Site_ID,Name,State,City,ARP_Latitude,ARP_Longitude,NPIAS_Category,CH_Latitude,CH_Longitude,Distance
0,50009.*A,Adak,Alaska,Adak Island,51.883583,-176.642483,Reliever,56.469252,-132.379680,1786.385658
1,50009.*A,Adak,Alaska,Adak Island,51.883583,-176.642483,Reliever,55.477123,-133.133806,1768.927706
2,50009.*A,Adak,Alaska,Adak Island,51.883583,-176.642483,Reliever,61.178048,-149.916588,1191.721753
3,50009.*A,Adak,Alaska,Adak Island,51.883583,-176.642483,Reliever,60.501690,-151.077002,1138.849769
4,50009.*A,Adak,Alaska,Adak Island,51.883583,-176.642483,Reliever,58.300164,-134.408460,1697.806402


In [100]:
#Filter down to combinations where distance is <1 miles from charger to airport
# This is a record of all ARP - CHA relationships where distance < 1 mile
Final_Joined = Joined_Set[Joined_Set['Distance']<1]

# A record of all airports where there's at least 1 charger
Final_Arps = Final_Joined.drop(columns = ['CH_Latitude','CH_Longitude','Distance']).drop_duplicates()

In [108]:
# Function takes in a selected airport, returns list of airports within 500 miles
def proximal_airports(arp_id):
    full_list = Final_Arps[Final_Arps.Site_ID != arp_id].copy()
    arp_loc = Final_Arps[Final_Arps.Site_ID == arp_id].copy()
    full_list['Distance'] = full_list.apply(lambda x : Get_Distance(x.ARP_Latitude,
                                                                    x.ARP_Longitude,
                                                                    arp_loc.ARP_Latitude,
                                                                    arp_loc.ARP_Longitude
                                                                   ),axis=1)
    proximal_list = full_list[full_list.Distance < 500]
    return proximal_list, arp_loc

In [109]:
arp_ids = list(Final_Arps.Site_ID)
prox_arps = list()

for ID in arp_ids:
    proximal_list, arp_loc = proximal_airports(ID)
    prox_arps.append(list(proximal_list.Site_ID))

Proximal_Sets = pd.DataFrame(columns = ['Airport', 'Prox_Airports'], data = zip(arp_ids, prox_arps))

In [110]:
from itertools import product
from sklearn.utils.extmath import cartesian

merge_me = Final_Arps.copy().drop(columns = ['NPIAS_Category'])
merge_me['key'] = 1

# Merge All Airport Combinations
All_Distances = pd.merge( merge_me, merge_me,  on = 'key').drop('key', axis = 1)
All_Distances.columns=['A_Site_ID', 'A_Name','A_State','A_City','A_Lat','A_Long',
                                     'B_Site_ID', 'B_Name','B_State','B_City','B_Lat','B_Long']

# Caluclate All Distances Between
All_Distances['Distance'] = All_Distances.apply(
    lambda x : Get_Distance(x.A_Lat, x.A_Long, x.B_Lat, x.B_Long),axis=1)


In [111]:
# Export all prepped datasets
airports.to_csv('prepped_data/airports_prepped.csv') # all airports
# NOTE: Chargers exported higher up in the notebook due to formatting needs
# i.e. had to drop charger city for analysis stuff, but wanted to keep it in the CSV export anyway
Final_Joined.to_csv('prepped_data/Final_Joined.csv') # all chargers - arp proximal relats 
                # (i.e. a record for each prox charger - ARP relationship)
Final_Arps.to_csv('prepped_data/Final_Arps.csv') # all arps w/ some proximal relationship
                # (i.e. a record for each individual airport in Final_Joined)
Proximal_Sets.to_csv('prepped_data/Proximal_Sets.csv') # All airport ID's within proximity of each airport
All_Distances.to_csv('prepped_data/All_Distances.csv')

In [112]:
All_Distances

,A_Site_ID,A_Name,A_State,A_City,A_Lat,A_Long,B_Site_ID,B_Name,B_State,B_City,B_Lat,B_Long,Distance
0,50385.*A,Juneau Intl,Alaska,Juneau,58.354711,-134.578469,50385.*A,Juneau Intl,Alaska,Juneau,58.354711,-134.578469,0.000000
1,50385.*A,Juneau Intl,Alaska,Juneau,58.354711,-134.578469,00128.*A,Anniston Rgnl,Alabama,Anniston,33.588167,-85.858111,2806.567774
2,50385.*A,Juneau Intl,Alaska,Juneau,58.354711,-134.578469,00146.*A,Auburn University Rgnl,Alabama,Auburn,32.615111,-85.434000,2875.071774
3,50385.*A,Juneau Intl,Alaska,Juneau,58.354711,-134.578469,00164.*A,Birmingham-Shuttlesworth Intl,Alabama,Birmingham,33.563889,-86.752306,2776.776898
4,50385.*A,Juneau Intl,Alaska,Juneau,58.354711,-134.578469,00438.*A,Mobile Rgnl,Alabama,Mobile,30.691417,-88.242833,2886.480501
...,...,...,...,...,...,...,...,...,...,...,...,...,...
81220,27759.*A,Cheyenne Rgnl/Jerry Olson Fld,Wyoming,Cheyenne,41.155639,-104.810472,27667.*A,Wausau Downtown,Wisconsin,Wausau,44.926283,-89.627003,808.778175
81221,27759.*A,Cheyenne Rgnl/Jerry Olson Fld,Wyoming,Cheyenne,41.155639,-104.810472,27682.*A,West Bend Muni,Wisconsin,West Bend,43.422167,-88.127917,865.902691
81222,27759.*A,Cheyenne Rgnl/Jerry Olson Fld,Wyoming,Cheyenne,41.155639,-104.810472,26627.*A,Elkins-Randolph County-Jennings Randolph Fld,West Virginia,Elkins,38.889444,-79.857139,1325.714783
81223,27759.*A,Cheyenne Rgnl/Jerry Olson Fld,Wyoming,Cheyenne,41.155639,-104.810472,26768.*A,Morgantown Muni-Walter L Bill Hart Fld,West Virginia,Morgantown,39.643594,-79.917547,1310.228646
